In [33]:
import pandas as pd
from IPython.display import display
import seaborn as sns
from matplotlib import pyplot as plt
import nltk
import re
from os import path
from unicodedata import normalize

%matplotlib inline

In [34]:
# de https://wiki.python.org.br/RemovedorDeAcentos
def remover_acentos(txt):
    return normalize("NFKD", txt).encode("ASCII", "ignore").decode("ASCII")

In [41]:
def clean_city_name(txt):
    sem_acento = remover_acentos(txt)
    clean = re.sub(r"\W+", "", sem_acento)
    return clean.lower()

Separamos os arquivos por cidade para facilitar a leitura:

In [43]:
for chunk_df in pd.read_csv(
    "../data/despesas/despesas-2019.csv",
    sep=";",
    encoding="windows-1254",
    chunksize=1000000,
):
    city_names = pd.unique(chunk_df["ds_municipio"])
    for name in city_names:
        city_df = chunk_df[chunk_df.ds_municipio == name]
        file_name = f"despesas-2019-{clean_city_name(name)}.csv"
        while path.exists(file_name):
            idx = 2
            file_name = file_name.replace(".csv", f"-{idx}.csv")
            idx += 1
        city_df.to_csv(f"../data/despesas/by_city/{file_name}")

/home/felipecgonc/.virtualenvs/deephack/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Não precisaremos juntar arquivos com *idx*, pois nenhum foi gerado! A pasta apresenta 644 arquivos no total.

In [11]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     /home/felipecgonc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/felipecgonc/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [121]:
df_csv = pd.read_csv(f"../data/despesas/by_city/despesas-2019-campinas.csv", sep=",")

In [122]:
df_csv["vl_despesa"] = (
    df_csv["vl_despesa"].apply(lambda x: float(x.replace(",", "."))).astype(float)
)

Aplicamos um método e uma biblioteca de NLP para definir as palavras mais recorrentes na descrição dos pagamentos.

In [123]:
top_N = 200
df = df_csv[df_csv.ds_municipio == "Campinas"]
df["historico_despesa"] = df["historico_despesa"].apply(
    lambda x: re.sub(r"\W+", " ", x)
)
df["historico_despesa"] = df["historico_despesa"].apply(
    lambda x: re.sub(r"^[a-zA-Z]+$", " ", x)
)
txt = df.historico_despesa.str.lower().str.cat(sep=" ")
words = nltk.tokenize.word_tokenize(txt)
word_dist = nltk.FreqDist(words)
stopwords = nltk.corpus.stopwords.words("portuguese")
words_except_stop_dist = nltk.FreqDist(w for w in words if w not in stopwords)
rslt = pd.DataFrame(
    words_except_stop_dist.most_common(top_N), columns=["Word", "Frequency"]
)
rslt = rslt[rslt.Word.str.isalpha()]

In [124]:
rslt.head()

,Word,Frequency
2,nº,21498
3,prot,20631
5,empenho,18769
6,pmc,16227
7,pagamento,15047


Em seguida, definimos as palvras que corresponderão a um gasto relativo ao ODS 11, de acordo com a especificação do mesmo. Notamos que incluímos versões sem acentos das palavras, para garantirmos que as descrições, às vezes feitas à mão, possam ser incluídas de forma mais abrangente possível.

# Metas do Objetivo 11

* **11.1** Até 2030, garantir o acesso de todos a habitação segura, adequada e a preço acessível, e aos serviços básicos e urbanizar as favelas	
* **11.2** Até 2030, proporcionar o acesso a sistemas de transporte seguros, acessíveis, sustentáveis e a preço acessível para todos, melhorando a segurança rodoviária por meio da expansão dos transportes públicos, com especial atenção para as necessidades das pessoas em situação de vulnerabilidade, mulheres, crianças, pessoas com deficiência e idosos	
* **11.3** Até 2030, aumentar a urbanização inclusiva e sustentável, e a capacidade para o planejamento e a gestão participativa, integrada e sustentável dos assentamentos humanos, em todos os países	
* **11.4** Fortalecer esforços para proteger e salvaguardar o patrimônio cultural e natural do mundo	
* **11.5** Até 2030, reduzir significativamente o número de mortes e o número de pessoas afetadas por catástrofes e diminuir substancialmente as perdas econômicas diretas causadas por elas em relação ao produto interno bruto global, incluindo os desastres relacionados à água, com o foco em proteger os pobres e as pessoas em situação de vulnerabilidade	
* **11.6** Até 2030, reduzir o impacto ambiental negativo per capita das cidades, inclusive prestando especial atenção à qualidade do ar, gestão de resíduos municipais e outros	
* **11.7** Até 2030, proporcionar o acesso universal a espaços públicos seguros, inclusivos, acessíveis e verdes, em particular para as mulheres e crianças, pessoas idosas e pessoas com deficiência	
* **11.a** Apoiar relações econômicas, sociais e ambientais positivas entre áreas urbanas, periurbanas e rurais, reforçando o planejamento nacional e regional de desenvolvimento	
* **11.b** Até 2020, aumentar substancialmente o número de cidades e assentamentos humanos adotando e implementando políticas e planos integrados para a inclusão, a eficiência dos recursos, mitigação e adaptação à mudança do clima, a resiliência a desastres; e desenvolver e implementar, de acordo com o Marco de Sendai para a Redução do Risco de Desastres 2015-2030, o gerenciamento holístico do risco de desastres em todos os níveis	
* **11.c** Apoiar os países menos desenvolvidos, inclusive por meio de assistência técnica e financeira, para construções sustentáveis e robustas, utilizando materiais locais

In [125]:
ods_11_1 = [
    "habitação",
    "habitacao",
    "habitacional",
    "favelas",
    "urbano",
    "urbanização",
    "urbanismo",
    "urbanizacao",
    "favela",
    "básicos",
]
ods_11_2 = [
    "transporte",
    "metrô",
    "metro",
    "onibus",
    "ônibus",
    "trem",
    "vlt",
    "monotrilho",
    "bonde",
    "frota",
    "trens",
    "transportes",
]
ods_11_3 = [
    "sustentável",
    "sustentavel",
    "sustentabilidade",
    "participativo",
    "participativa",
    "assentamentos",
]
ods_11_4 = [
    "partimônio",
    "cultura",
    "restauração",
    "patrimonio",
    "restauracao",
    "reserva",
    "natural",
    "natureza",
    "cultural",
    "patrimonial",
    "meio-ambiente",
]
ods_11_5 = [
    "mortalidade",
    "catástrofe",
    "catastrofe",
    "catástrofes",
    "catastrofes",
    "acidente",
    "acidentes",
    "enchentes",
    "enchente",
    "deslizamento",
    "desastre",
    "desastres",
    "chuva",
    "chuvas",
    "barragem",
    "contaminação",
    "despoluição",
    "contaminacao",
    "despoulicao",
    "contaminado",
    "contaminada",
]
ods_11_6 = [
    "ar",
    "poluição",
    "resíduos",
    "poluicao",
    "residuos",
    "esgoto",
    "esgotos",
    "saneamento",
    "coleta",
    "lixo",
    "lixão",
    "lixao",
    "lixoes",
    "lixões",
    "aterro",
    "aterros",
    "reciclagem",
    "reciclado",
    "recicla",
    "descarte",
]
ods_11_7 = [
    "espaço",
    "espaco",
    "acessível",
    "acessibilidade",
    "convivência",
    "convivencia",
    "conviver",
    "acessivel",
    "cadeirante",
    "cego",
    "deficiente",
    "deficientes",
    "deficiencia",
    "deficiência",
    "idoso",
    "idosos",
    "idosa",
    "mulher",
    "mulheres",
    "verde",
    "verdes",
]
ods_11_a = ["rural", "campo", "periurbana"]
ods_11_b = [
    "eficiência",
    "eficiencia",
    "clima",
    "climático",
    "climatico",
    "aquecimento",
    "sendai",
]
ods_11_c = []

ods_11 = (
    ods_11_1
    + ods_11_2
    + ods_11_3
    + ods_11_4
    + ods_11_5
    + ods_11_6
    + ods_11_7
    + ods_11_a
    + ods_11_b
    + ods_11_c
)

Em seguida, checamos quais das palavras mais presentes nas despesas correspondem a um gasto relativo à meta do ODS 11.

In [126]:
rslt["ods"] = rslt["Word"].apply(lambda x: x in ods_11)

In [127]:
ods_rslt = rslt[rslt.ods]

E colocamos as palavras que efetivamente foram encontradas em uma lista.

In [128]:
ods_list = list(ods_rslt["Word"])

Em seguida, achamos esses gastos no DataFrame original, para determinarmos o valor total gasto com o ODS 11.

In [120]:
def is_ods(txt, ods_list):
    is_ods = False
    for ods in ods_list:
        if ods in txt:
            is_ods = True
    return is_ods

In [129]:
df_csv["is_ods"] = df_csv["historico_despesa"].apply(lambda x: is_ods(x, ods_list))

In [131]:
df_ods = df_csv[df_csv.is_ods]

In [133]:
# Soma total gasta com ODS
ods_sum = sum(df_ods["vl_despesa"])

# Soma total
total_sum = sum(df_csv["vl_despesa"])

O valor gasto com o ODS 11 foi de:

In [134]:
ods_sum

120674605.77999994

Que corresponde a: % do gasto

In [135]:
round(ods_sum / total_sum, 3) * 100

1.0999999999999999

Tendo a metodologia estabelecida, transformamos essa visualização em uma tela do Dashboard usando o streamlit.